In [1]:
from time import sleep, strftime
from time import sleep
from ib.ext.Contract import Contract
from ib.opt import Connection, message

class HistoricalDataCollection:
    def __init__(self, contract, date, t, duration):
        self.contract = contract
        self.date = date
        self.t = t
        self.duration = str(duration) + ' S'
        self.tick_id = 3
        self.con = Connection.create(port=4002, clientId=2707)
        self.con.register(self.process_data, message.historicalData)
        self.con.connect()
        sleep(1)
        end_datetime = ('%s %s US/Eastern' % (self.date, self.t))
        self.con.reqHistoricalData(tickerId=self.tick_id, contract=self.contract,\
                                   endDateTime=end_datetime,durationStr=self.duration, barSizeSetting='5 secs',\
                                   whatToShow='MIDPOINT', useRTH=1, formatDate=1)
        sleep(1)
        self.data_received = False

    def close(self):
        print("closing...")
        while not self.data_received:
            pass
        print("received")
        self.con.cancelHistoricalData(self.tick_id)
        sleep(1)
        self.con.disconnect()
        sleep(1)
    def disconnect(self):
        self.con.disconnect()

    def process_data(self, msg):
        print("processing")
        print(msg)
        if msg.open != -1:
            print(msg.date, msg.open, msg.high, msg.low, msg.close, msg.volume, msg.count, msg.WAP, msg.hasGaps)
        else:
            self.data_received = True

def makeOptionContract(symbol,expiry,strike,option_type):
    newContract = Contract()
    newContract.m_symbol = symbol
    newContract.m_secType = "OPT"
    newContract.m_exchange = "SMART"
    newContract.m_currency = "USD"
    newContract.m_expiry = expiry
    newContract.m_strike = strike
    newContract.m_right = option_type
    return newContract
def makeStockContract(symbol):
    newContract = Contract()
    newContract.m_symbol = symbol
    newContract.m_secType = "STK"
    newContract.m_exchange = "NASDAQ"
    newContract.m_currency = "USD"
    return newContract
def makeCashContract(currency):
    newContract = Contract()
    newContract.m_symbol = "USD"
    newContract.m_secType = "CASH"
    newContract.m_exchange = "SMART"
    newContract.m_currency = currency
    return newContract

if __name__ == "__main__":
    duration = 60
    t = '11:15:00'
    date = "20161003"
    contract = makeStockContract('AAPL')
    broker = HistoricalDataCollection(contract, date, t, duration)
    broker.close()
